In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.express as px
import plotly.figure_factory as ff
from arch import arch_model

from ipywidgets import HBox, VBox, Dropdown, Output
from scipy.optimize import fmin, minimize
from scipy.stats import t
from scipy.stats import norm
from math import inf
from IPython.display import display

import bs4 as bs
import requests
import yfinance as yf
import datetime

In [2]:
def vecl(matrix):
    lower_matrix = np.tril(matrix,k=-1)
    array_with_zero = np.matrix(lower_matrix).A1

    array_without_zero = array_with_zero[array_with_zero!=0]

    return array_without_zero

In [3]:
def garch_t_to_u(rets, res):
    mu = res.params['mu']
    nu = res.params['nu']
    est_r = rets - mu
    h = res.conditional_volatility
    std_res = est_r / h
    # we could also just use:
    # std_res = res.std_resid
    # but it's useful to see what is going on
    udata = t.cdf(std_res, nu)
    return udata

In [4]:
def loglike_norm_dcc_copula(theta, udata):
    N, T = np.shape(udata)
    llf = np.zeros((T,1))
    trdata = np.array(norm.ppf(udata).T, ndmin=2)
    
    
    Rt, veclRt =  dcceq(theta,trdata)

    for i in range(0,T):
        llf[i] = -0.5* np.log(np.linalg.det(Rt[:,:,i]))
        llf[i] = llf[i] - 0.5 *  np.matmul(np.matmul(trdata[i,:] , (np.linalg.inv(Rt[:,:,i]) - np.eye(N))) ,trdata[i,:].T)
    llf = np.sum(llf)

    return -llf

In [5]:
def dcceq(theta,trdata):
    T, N = np.shape(trdata)

    a, b = theta
    
    if min(a,b)<0 or max(a,b)>1 or a+b > .999999:
        a = .9999 - b
        
    Qt = np.zeros((N, N ,T))

    Qt[:,:,0] = np.cov(trdata.T)

    Rt =  np.zeros((N, N ,T))
    veclRt =  np.zeros((T, int(N*(N-1)/2)))
    
    Rt[:,:,0] = np.corrcoef(trdata.T)
    
    for j in range(1,T):
        Qt[:,:,j] = Qt[:,:,0] * (1-a-b)
        Qt[:,:,j] = Qt[:,:,j] + a * np.matmul(trdata[[j-1]].T, trdata[[j-1]])
        Qt[:,:,j] = Qt[:,:,j] + b * Qt[:,:,j-1]
        Rt[:,:,j] = np.divide(Qt[:,:,j] , np.matmul(np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2)).T , np.sqrt(np.array(np.diag(Qt[:,:,j]), ndmin=2))))
    
    for j in range(0,T):
        veclRt[j, :] = vecl(Rt[:,:,j].T)
    return Rt, veclRt

In [6]:
def run_garch_on_return(rets, udata_list, model_parameters):
    for x in rets:
        am = arch_model(rets[x], dist = 't')
        short_name = x.split()[0]
        model_parameters[short_name] = am.fit(disp='off')
        udata = garch_t_to_u(rets[x], model_parameters[short_name])
        udata_list.append(udata)
    return udata_list, model_parameters

In [7]:
def update_corr_data(change):
    a1corr = rets.loc[:, pair_dropdown.value.split('-')].corr().values[0][1]
    a1dcc = pd.DataFrame(veclRt[:,corr_name_list.index(pair_dropdown.value)],index = rets.iloc[:,:5].dropna().index)
    a1dcc.columns = ['DCC']
    a1dcc['corr'] = a1corr
    corr_line_plot = px.line(a1dcc, title = 'DCC vs unconditional correlation for ' + pair_dropdown.value, width=1000, height=500)
    output_graphics.clear_output()
    with output_graphics:
        display(corr_line_plot)

In [9]:
rets = pd.read_csv('Compiled_Data.csv').set_index('Date')
rets = rets[["USTC","FRAX","AMPL","DAI","EOSDT","USDC","USDT","PAXG","XAUT"]].dropna(how='all')
rets

,USTC,FRAX,AMPL,DAI,EOSDT,USDC,USDT,PAXG,XAUT
Date,,,,,,,,,
01-11-2021,1.001808,0.997458,1.878852,1.000939,0.995157,0.999861,1.000291,1791.763428,1795.770142
02-11-2021,1.002310,0.999233,1.869730,1.001310,0.940047,0.999896,1.000815,1791.566650,1795.850586
03-11-2021,1.004410,1.003389,1.756711,1.000819,0.816678,1.000225,1.000678,1779.253296,1787.464844
04-11-2021,1.003888,0.998294,1.520222,1.000081,1.009842,1.000206,1.000857,1793.724976,1800.415894
05-11-2021,1.003383,1.004354,1.548847,1.001021,0.891854,1.000562,1.001553,1819.751953,1824.858154
...,...,...,...,...,...,...,...,...,...
26-09-2022,0.033281,0.993318,1.170409,0.999868,0.895347,1.000165,0.999990,1619.209106,1626.883545
27-09-2022,0.031270,0.992912,1.095596,0.999813,0.882125,1.000123,0.999980,1621.632813,1629.180786
28-09-2022,0.031109,0.993726,1.155449,0.999388,0.893674,1.000011,0.999976,1648.627930,1654.329590


In [18]:
px.line(rets[["XAUT"]], title="Tether Gold (XAUT)", width=1000, height=400).show()

In [60]:
px.line(rets[["FRAX","DUSD"]], title='Algorithmic', width=1000, height=500).show()

In [61]:
px.line(rets[["DAI"]], title='Crypto-Backed', width=1000, height=500).show()

In [62]:
px.line(rets[["USDC","USDT","BUSD","GUSD"]], title='Fiat-Backed', width=1000, height=500).show()

In [63]:
px.line(rets[["PAXG","XAUT"]], title='Gold-Backed', width=1000, height=500).show()

In [19]:
model_parameters = {}
udata_list = []
udata_list, model_parameters = run_garch_on_return(rets.dropna(), udata_list, model_parameters)
print(model_parameters)

C:\Users\sadas\Anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 5.785e-06. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 1000 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


C:\Users\sadas\Anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning:

y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.04113. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.


C:\Users\sadas\Anaconda3\lib\site-packages\arch\univariate\base.py:310: DataScaleWarning:

y is poorly scale

{'USTC':                         Constant Mean - GARCH Model Results                         
Dep. Variable:                         USTC   R-squared:                       0.000
Mean Model:                   Constant Mean   Adj. R-squared:                  0.000
Vol Model:                            GARCH   Log-Likelihood:                186.655
Distribution:      Standardized Student's t   AIC:                          -363.310
Method:                  Maximum Likelihood   BIC:                          -344.469
                                              No. Observations:                  320
Date:                      Fri, Dec 16 2022   Df Residuals:                      319
Time:                              13:18:09   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
------------------------------------------------------------------------

In [20]:
cons = ({'type': 'ineq', 'fun': lambda x:  -x[0]  -x[1] +1})
bnds = ((0, 0.5), (0, 0.9997))

%time opt_out = minimize(loglike_norm_dcc_copula, [0.01, 0.95], args = (udata_list), bounds=bnds, constraints=cons)

Wall time: 5.49 s


In [21]:
print(opt_out.success)
print(opt_out.x)

True
[0.23623014 0.69291251]


In [22]:
llf  = loglike_norm_dcc_copula(opt_out.x, udata_list)
llf

-1176.3110557632046

In [23]:
trdata = np.array(norm.ppf(udata_list).T, ndmin=2)
Rt, veclRt = dcceq(opt_out.x, trdata)

stock_names = [x.split()[0] for x in rets.columns]
corr_name_list = []
for i, name_a in enumerate(stock_names):
    if i == 0:
        pass
    else:
        for name_b in stock_names[:i]:
            corr_name_list.append(name_a + "-" + name_b)

In [24]:
garch_vol_df = pd.concat([pd.DataFrame(model_parameters[x].conditional_volatility/100)*1600 for x in model_parameters], axis=1)
garch_vol_df.columns = stock_names

In [26]:
stock1,stock2 = ["USTC","USDT"]
px.scatter(garch_vol_df, x = stock1, y=stock2, width=1000, height=500, title='GARCH Volatility').show()

In [27]:
px.line(garch_vol_df[["USTC","FRAX","AMPL","DAI","EOSDT","USDC","USDT","PAXG","XAUT"]], title='GARCH Conditional Volatility', width=1000, height=500).show()

In [72]:
px.line(garch_vol_df[["PAXG","XAUT"]], title='GARCH Conditional Volatility', width=1000, height=500).show()

In [39]:
dcc_corr = pd.DataFrame(veclRt, index = rets.iloc[:,:5].dropna().index, columns= corr_name_list)
dcc_plot = px.line(dcc_corr['XAUT-USTC'], title = 'Dynamic Conditional Correlation of USTC with XAUT', width=1000, height=400)
dcc_plot.show()

In [74]:
output_graphics = Output()
pair_dropdown = Dropdown(options=[''] + corr_name_list)
pair_dropdown.observe(update_corr_data, 'value')
VBox([pair_dropdown, output_graphics])